# Preparación

In [ ]:
import numpy as np
import csv
import math
import random
import time
import sys
import pandas as pd
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ipywidgets
# To enable `ipywidgets`
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
from ipywidgets import widgets
from IPython.display import display

In [ ]:
!pip install -q gwpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Esta librería nos va a permitir ver el tiempo de ejecución de todas las celdas en el cuaderno.

In [ ]:
%%capture
!pip install ipython-autotime
%load_ext autotime

time: 693 µs (started: 2023-06-19 20:30:31 +00:00)


# Business Understanding

El objetivo del Business Understanding es determinar los objetivos, requisitos y condiciones del proyecto. Parte desde conversaciones con la empresa para conseguir esta información por medio de técnicas de capturas de requisitos. Una vez obtenidos los objetivos de la empresa en terminología empresarial, se debe de analizar la situación y, finalmente, determinar cuáles van a ser los objetivos que va a tener la minería de datos.

Esta fase no tiene código asociado, pues trata de entender el ámbito del problema antes de empezar a analizar los datos aportados.
from sklearn.model_selection import train_test_split

# Data Understanding

La fase de Data Understanding tiene como objetivo principal comprender a fondo los datos disponibles y establecer una base sólida para el futuro análisis y modelado. Al obtener un conocimiento profundo de los datos, se pueden tomar decisiones informadas en futuras decisiones como la técnica de minería de datos o la preparación de datos.

## Recolección de datos

Todos los conjuntos de datos están disponibles en Google Drive

In [ ]:
dropdown = widgets.Dropdown(options =["DoS", "Fuzzy", "Gear", "RPM"])
output = widgets.Output()
path = '/content/drive/MyDrive/tfg/datasets/DoS_dataset.csv'

def select(change):
  global path
  output.clear_output()
  if(change.new == 'DoS'):
    with output:
      path = '/content/drive/MyDrive/tfg/datasets/DoS_dataset.csv'
  elif(change.new == 'Fuzzy'):
    with output:
      path = '/content/drive/MyDrive/tfg/datasets/Fuzzy_dataset.csv'
  elif(change.new == 'Gear'):
    with output:
      path = '/content/drive/MyDrive/tfg/datasets/Gear_dataset.csv'
  elif(change.new == 'RPM'):
    with output:
      path = '/content/drive/MyDrive/tfg/datasets/RPM_dataset.csv'
  else:
    with output:
      display("Error: option not found")

dropdown.observe(select, names="value")
display(dropdown)
display(output)

Dropdown(options=('DoS', 'Fuzzy', 'RPM', 'Gear'), value='DoS')

Output()

time: 35.9 ms (started: 2023-06-19 20:30:31 +00:00)


In [ ]:
columns = ['Tiempo', 'ID', 'DLC', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'Etiqueta']
df = pd.read_csv(path, delimiter=',', header=None, names=columns)

time: 12.3 s (started: 2023-06-19 20:30:31 +00:00)


## Descripción de datos

Vamos a ver cuántos mensajes son reales (R) y cuántos de ellos malintencionados (T).

In [ ]:
%%capture
def getStats(df):
  print(len(df.index))
  print(df['Etiqueta'].value_counts())

time: 465 µs (started: 2023-06-19 20:30:43 +00:00)


In [ ]:
getStats(df)

3665771
R    3047062
T     587521
Name: Etiqueta, dtype: int64
time: 395 ms (started: 2023-06-19 20:30:43 +00:00)


## Exploración de datos

Vamos a ver qué tipos de datos tiene nuestro conjunto de datos.

In [ ]:
print(df.dtypes)

Tiempo      float64
ID           object
DLC           int64
M1           object
M2           object
M3           object
M4           object
M5           object
M6           object
M7           object
M8           object
Etiqueta     object
dtype: object
time: 2.36 ms (started: 2023-06-19 20:30:44 +00:00)


In [ ]:
df.head()

,Tiempo,ID,DLC,M1,M2,M3,M4,M5,M6,M7,M8,Etiqueta
0,1.478198e+09,0316,8,05,21,68,09,21,21,00,6f,R
1,1.478198e+09,018f,8,fe,5b,00,00,00,3c,00,00,R
2,1.478198e+09,0260,8,19,21,22,30,08,8e,6d,3a,R
3,1.478198e+09,02a0,8,64,00,9a,1d,97,02,bd,00,R
4,1.478198e+09,0329,8,40,bb,7f,14,11,20,00,14,R


time: 41.7 ms (started: 2023-06-19 20:30:44 +00:00)


### Densidad de mensajes malintencionados

Para conseguir la densidad de mensajes malintencionados, vamos a agruparlos según las marcas de tiempo y mostrarlo en forma de gráfico. Esto se conseguiría con la siguiente función:

```
def getMessageConcentration(df):
  df_R = df.copy(deep=True)
  df_R.set_index('Tiempo', inplace=True)

  df_R = df_R.groupby(pd.Grouper(freq='U')).mean()

  df_R['Etiqueta'].plot()
  plt.show()
```

In [ ]:
def getMessageConcentration(df):
  df_R = df.copy(deep=True)
  df_R.set_index('Tiempo', inplace=True)

  df_R = df_R.groupby(pd.Grouper(freq='U')).mean()

  df_R['Etiqueta'].plot()
  plt.show()

### Patrones superficiales en los mensajes malintencionados

In [ ]:
fakeMessages = df[df['Etiqueta'] == 0]
fakeMessages

,Tiempo,ID,DLC,M1,M2,M3,M4,M5,M6,M7,M8,Etiqueta


time: 441 ms (started: 2023-06-19 20:30:44 +00:00)


## Transformación de tipos

Las transformaciones implementadas aquí están explicadas en el documento.

In [ ]:
def transformData(df):
  df = df.fillna(0)
  df = df.apply(normalizeMessages, axis=1)
  df[['ID', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8']] = df[['ID', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8']].applymap(lambda cell : int(str(cell), 16))
  df['Etiqueta'] = df['Etiqueta'].apply(lambda cell : 1 if cell == 'R' else 0)
  df['Delta'] = df['Tiempo'].diff()
  df.tail(1)['Delta'] = df[['Delta']].mean()
  df['Delta'] = df['Delta'].apply(np.float32)
  df['Tiempo'] = pd.to_datetime(df['Tiempo'], unit='us')
  df = df.fillna(0)

  return df

def normalizeMessages(row):
  dlc = row['DLC']
  if dlc < 8:
      messageIndex = 'M' + str(int(dlc + 1))
      row['Etiqueta'] = row[messageIndex]
      row[messageIndex] = 0

  return row

time: 3.82 ms (started: 2023-06-19 20:30:44 +00:00)


In [ ]:
df = transformData(df)
print(df.dtypes)

<ipython-input-14-3e4bbd79c47c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.tail(1)['Delta'] = df[['Delta']].mean()


Tiempo      datetime64[ns]
ID                   int64
DLC                  int64
M1                   int64
M2                   int64
M3                   int64
M4                   int64
M5                   int64
M6                   int64
M7                   int64
M8                   int64
Etiqueta             int64
Delta              float32
dtype: object
time: 3min 36s (started: 2023-06-19 20:30:44 +00:00)


Gracias a esta celda, podemos comprobar que para los conjuntos de ataques DoS y Spoofing, los mensajes malintencionados son exactamente iguales entre sí menos las marcas de tiempo.

## Correlación de variables

Para obtener la correlación entre variables del conjunto de datos, se ejecutaría el siguiente código:
```
def getCorrelation(df):
	  var_list = ['Tiempo', 'ID', 'DLC', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'Delta']
	  coef_matrix = numpy.zeros(shape=(len(var_list), len(var_list)))
	  for var1, var2 in list(itertools.combinations(enumerate(var_list), 2)):
	      coef_matrix[var1[0]][var2[0]] = df[var1[1]].corr(df[var2[1]])

	  plt.figure(figsize=(10, 10))
	  sns.set(font_scale=1.5)

	  sns.heatmap(coef_matrix,
	              cmap='coolwarm',
	              annot=False,
	              fmt='.5g',
	              vmax=1,
	              vmin=-1)

	  plt.xlabel('Predicted',fontsize=22)
	  plt.ylabel('Actual',fontsize=22)
	  plt.show()

```

# Data Preparation

La fase de Data Preparation, como dice su nombre, consiste en utilizar los procesos necesarios para dejar los datos preparados para las siguientes fases. El objetivo de esta fase es realizar todas las transformaciones necesarias que se consideren necesarias para producir uno o más conjuntos de datos preparados para la fase de modelado; así como añadir nuevos conjuntos de datos para enriquecer la población.

## Selección de datos

### Exclusión de datos

Vamos a realizar un muestreo sistemático y un muestreo estratificado.

In [ ]:
def stratifiedSampling(df, percentage):
  normalMessages = df[df['Etiqueta'] == 1]
  fakeMessages = df[df['Etiqueta'] == 0]

  sizeNormal = int((len(fakeMessages) * (1 - percentage))/percentage)
  normalMessages = normalMessages.sample(n=sizeNormal, replace=False, random_state=42)

  stratifiedSample = pd.concat([fakeMessages, normalMessages])

  df_stratified = stratifiedSample.reset_index(drop=True)

  return df_stratified

time: 994 µs (started: 2023-06-19 20:34:20 +00:00)


Para el muestreo sistemático, vamos a permitir solo los grupos con una concentración mayor al 10% de mensajes malintencionados.

Para el muestreo estratificado, vamos a muestrear el conjunto para que el porcentaje de mensajes malintencionados sea del 30%.

In [ ]:
df_stratified = stratifiedSampling(df, 0.3)

time: 755 ms (started: 2023-06-19 20:34:20 +00:00)


### Exportación de muestras

In [ ]:
def writeDataset(df, original_path, sample_type=None):
  path = original_path.split('/')[-1].split('_')[0] + '_' + str(sample_type) + '.csv'
  df.to_csv(path, index=False)

time: 436 µs (started: 2023-06-19 20:34:21 +00:00)


In [ ]:
writeDataset(df_stratified, path, 'stratified')

time: 13.2 s (started: 2023-06-19 20:34:21 +00:00)
